In [28]:
import redpitaya_scpi as scpi
import numpy as np
import time


In [29]:
# Red Pitaya Stuff

address = "rp-f06897.local"
rp = scpi.scpi(address)
dec_exp = np.array([0,3,6,10,13,16])
dec = 2**dec_exp
t_total = np.array([131.072e-6, 1.049e-3, 8.389e-3, 134.218e-3,1.074, 8.590])
dec

array([    1,     8,    64,  1024,  8192, 65536], dtype=int32)

ASCII Version

In [39]:
tn = [time.time()]
rp.tx_txt('ACQ:RST')
rp.tx_txt('ACQ:DATA:FORMAT ASCII')
rp.tx_txt("ACQ:DEC 1024")
rp.tx_txt('ACQ:START')
rp.tx_txt('ACQ:TRIG NOW')

while 1:
    rp.tx_txt('ACQ:TRIG:STAT?')
    if rp.rx_txt() == 'TD':
        break
        
rp.tx_txt('ACQ:SOUR1:DATA?')
buf = rp.rx_txt()
tn.append(time.time())

print("Total time: %f" % (tn[-1]-tn[0]))
print("Splits:")
for i in range(len(tn)-1):
    print("%f" % (tn[i+1]-tn[i]))

print('\nBuffer: \n')
print(buf)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x86 in position 5: invalid start byte

Binary version

In [37]:
tn = [time.time()]
rp.tx_txt('ACQ:RST')
rp.tx_txt("ACQ:DEC 1024")
rp.tx_txt('ACQ:DATA:FORMAT BIN')
rp.tx_txt('ACQ:START')
rp.tx_txt('ACQ:TRIG NOW')
rp.tx_txt('ACQ:SOUR1:DATA?')

buf = rp.rx_arb()
tn.append(time.time())

print("Total time: %f" % (tn[-1]-tn[0]))
print("Splits:")
for i in range(len(tn)-1):
    print("%f" % (tn[i+1]-tn[i]))

print('\nBuffer: \n')
print(buf)

Total time: 0.001000
Splits:
0.001000

Buffer: 

False


If I run the above code, my red pitaya then also seems to completely crash...its built-in webserver is no longer responsive.

And all subsequent attempst to connect with ASCII code above also fail until I manage to stop and restart the SCPI server?